# Projeto Robo Seguidor de Linha usando OpenCV
## Type Robot: Robo OBR Resgate
### Comunidade: BigRobot  
#### Projetistas: Debora, Juan e Daniel


### Sumario 

1. Instalação do Ubuntu Mate
2. Instalação e configuração do OpenCV 
3. Instalação do Arduino
4. Programando OpenCV usando Simulação
5. Circuito eletronico do Robo
6. Programando no arduino
7. Topologia da rede
8. Usando SSH para acesso remoto no robo




## Instalanção do Ubuntu Mate

##  Instalação e configuração do OpenCV
### Dependencias necessarias 

terminal@terminal$ sudo apt-get install -y build-essential cmake pkg-config

terminal@terminal$ sudo apt-get install -y libjpeg-dev libtiff5-dev libjasper-dev libpng12-dev

terminal@terminal$ sudo apt-get install -y libavcodec-dev libavformat-dev libswscale-dev libv4l-dev

terminal@terminal$ sudo apt-get install -y libxvidcore-dev libx264-dev

terminal@terminal$ sudo apt-get install -y libgtk2.0-dev

terminal@terminal$ sudo apt-get install -y libatlas-base-dev gfortran

terminal@terminal$ sudo apt-get install -y python2.7-dev python3-dev


#### Faça Download da opencv usando wget ainda no terminal, depois usando unzip extraia 

#### terminal@terminal $ wget -O opencv.zip https://github.com/Itseez/opencv/archive/3.1.0.zip

#### terminal@terminal $ unzip opencv.zip


### Faça download da opencv contrib e extraia usando unzip 

#### terminal@terminal $ wget -O opencv_contrib.zip https://github.com/Itseez/opencv_contrib/archive/3.1.0.zip

#### terminal@terminal $ unzip opencv_contrib.zip

#### Use o pip para instalar a biblioteca numpy no python

#### terminal@terminal $ pip install numpy

### Inicie o processo de compilar o OpenCV 

#### terminal@terminal $ cd ~

#### terminal@terminal $ cd ~/opencv-3.1.0/

#### terminal@terminal $ mkdir build

#### terminal@terminal $ cd build

-  terminal@terminal $ cmake -D CMAKE_BUILD_TYPE=RELEASE \
    -D CMAKE_INSTALL_PREFIX=/usr/local \    
    -D INSTALL_PYTHON_EXAMPLES=ON \    
    -D OPENCV_EXTRA_MODULES_PATH=~/opencv_contrib-3.1.0/modules \    
    -D BUILD_EXAMPLES=ON \    
    -DENABLE_PRECOMPILED_HEADERS=OFF .. #test lesk test ..
    


#### terminal@terminal $ make -j2

#### terminal@terminal $ sudo make install

#### terminal@terminal $ sudo ldconfig


## Instalação do Arduino

## Programando OpenCV usando Simulação 

#### Importando bibliotecas 

In [2]:
import cv2
import numpy as np

#### Valores de ajustes empiricos

In [3]:
LimiarBinarizacao = 125       #este valor eh empirico. Ajuste-o conforme sua necessidade 
AreaContornoLimiteMin = 5000  #este valor eh empirico. Ajuste-o conforme sua necessidade 

#### Função de Tratar Imagem há entrada do parametro é a captura do Frame enviado pela webcam, a função gera uma saida de > 0: direita; se for retorno < 0: esquerda; se retornar 0 deve ficar parado.

In [9]:
#Funcao: trata imagem e retorna se o robo seguidor de linha deve ir para a esqueda ou direita
#Parametros: frame capturado da webcam e primeiro frame capturado
#Retorno: < 0: robo deve ir para a direita
#         > 0: robo deve ir para a esquerda
#         0:   nada deve ser feito
def TrataImagem(img):
    #obtencao das dimensoes da imagem
    height = np.size(img,0)
    width= np.size(img,1)
    QtdeContornos = 0
    DirecaoASerTomada = 0
    
    #tratamento da imagem
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
    FrameBinarizado = cv2.threshold(gray,LimiarBinarizacao,255,cv2.THRESH_BINARY)[1]
    FrameBinarizado = cv2.dilate(FrameBinarizado,None,iterations=2)
    FrameBinarizado = cv2.bitwise_not(FrameBinarizado)
    
    _, cnts, _ = cv2.findContours(FrameBinarizado.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(img,cnts,-1,(255,0,255),3)

    for c in cnts:
        if cv2.contourArea(c) < AreaContornoLimiteMin:
            continue
            QtdeContornos = QtdeContornos + 1

        (x, y, w, h) = cv2.boundingRect(c)                                  
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)    
        CoordenadaXCentroContorno = (x+x+w)/2
        CoordenadaYCentroContorno = (y+y+h)/2
        PontoCentralContorno = (CoordenadaXCentroContorno,CoordenadaYCentroContorno)
        cv2.circle(img, PontoCentralContorno, 1, (0, 0, 0), 5)        
        DirecaoASerTomada = CoordenadaXCentroContorno - (width/2)   #em relacao a linha central
     
    #output da imagem
    #linha em azul: linha central / referencia
    #linha em verde: linha que mostra distancia entre linha e a referencia
    cv2.line(img,(width/2,0),(width/2,height),(255,0,0),2)
    
    if (QtdeContornos > 0):
        cv2.line(img,PontoCentralContorno,(width/2,CoordenadaYCentroContorno),(0,255,0),1)
    
    cv2.imshow('Analise de rota DA OBR',img)
    cv2.waitKey(10)
    return DirecaoASerTomada, QtdeContornos




#### Aqui setamos o programa para fazer captura dos dados da simulação.mp4, um dos motivos para  rodar o for é calibrar a entrada da camera

In [10]:
#Programa principal


camera = cv2.VideoCapture("simulacao.mp4")
camera.set(3,320)
camera.set(4,240)

#faz algumas leituras de frames antes de consierar a analise
#motivo: algumas camera podem demorar mais para se "acosumar a luminosidade" quando ligam, capturando frames consecutivos com muita variacao de luminosidade. Para nao levar este efeito ao processamento de imagem, capturas sucessivas sao feitas fora do processamento da imagem, dando tempo para a camera "se acostumar" a luminosidade do ambiente
for i in range(0,20):
    (grabbed, Frame) = camera.read()

#### Inicio do Loop capturando frame pela função do metodo camera.read(), após a entrada de TrataImagem ele gera uma quantidade de linhas e uma direção 
* se Direcão > 0: Direita
* se DirecãO < 0: Esquerda
* se Direcão == 0: Exatamente na referencia

In [12]:
while True:
    try:
      (grabbed, Frame) = camera.read()
      if cv2.waitKey(1) & 0xFF == ord('q'):
            break
      if (grabbed):
          Direcao,QtdeLinhas = TrataImagem(Frame)
          if (QtdeLinhas == 0):
             print ("Nenhuma linha encontrada. O robo ira parar.")
             continue
        
          if (Direcao > 0):
              print ("Distancia da linha de referencia: "+str(abs(Direcao))+" pixels a [DIREITA]")
          if (Direcao < 0):
              print ("Distancia da linha de referencia: "+str(abs(Direcao))+" pixels a [ESQUERDA]")      
          if (Direcao == 0):
              print ("Exatamente na linha de referencia!")
    except (KeyboardInterrupt):
          print ("STOP MODA FOCA")
          break;
exit(1)   


STOP MODA FOCA


### Video de test 
#### PS:  Este video de teste tem como objetivo de testar o sistema de visão computacional antes de rodar ele direto na pista para ver como o algoritmo se comporta


In [21]:
%%HTML 
<video width="320" height="240" controls><source src="simulacao.mp4" type="video/mp4"></video>
<video width="320" height="240" controls><source src="simulacao_opencv.mp4" type="video/mp4"></video>

## Circuito eletronico do Robo

* 1x Arduino 
* 2x Motores DC 
* 4X Modulo rele 
* 1x Raspberry
* 1x Camera USB
* 1x PowerBank 5v
* 1x Bateria 6v Chumbo


In [1]:
Image("../")


NameError: name 'Image' is not defined

## Programando no arduino


## Topologia da rede


## Usando SSH para acesso remoto no robo